# TESTING THE DATASETS AND TASKS ON CHATGPT WITH OUR PROMPTS

In [ ]:
!pip install openai
!pip install tiktoken
import openai
import tiktoken
encoding = tiktoken.get_encoding("gpt2")
with open('/content/drive/MyDrive/DS-Environment-Project/climate-change-api-key.txt', 'r') as file:
    key = file.readline()
openai.api_key = key

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Using cached openai-0.27.8-py3-none-any.whl (73 kB)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 39.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 114.5/114.5 kB 14.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 28.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 149.6/149.6 kB 20.2 MB/s eta 0:00:00
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 61.5 MB/s eta 0:00:00


In [ ]:
import json
import time
def batch_gpt(prompt,target_texts, batch_size):
    l = len(target_texts)
    size = int(l / batch_size) + 1
    size = batch_size
    text_list = np.array_split(target_texts, 3)
    print(f"Total records {l}, number of chunks = {size}")
    rez_keys = []
    rez_vals = []
    rez = []
    for i, texts in enumerate(text_list):
        text = "\n".join(texts)
        p = prompt + text
        print("prompt", p)
        response = openai.ChatCompletion.create(
            model="gpt-3.5-turbo",
            messages=[{"role": "user", "content": p}]
        )
        r = response["choices"][0]["message"]["content"]
        print(result, r)
        rez.append(r)
        try:
            dictData= json.loads(r)
            keys = list(dictData.keys())
            values = list(dictData.values())
            rez_keys += keys
            rez_vals += values
        except:
            print("error parsing"+r)
        print(f"i={i + 1}: shape:{len(rez_keys)}")
    return rez_keys, rez_vals, rez
def batch_gpt_len(prompt,target_texts, batch_size):
    l = len(target_texts)
    rez_keys = []
    rez_vals = []
    rez = []
    i =0
    while i < l:
        text = ""
        for j in range(batch_size):
            if i<l:
                if batch_size > 1:
                    text += f"{i}. "+target_texts[i]+"\n\n"
                else:
                    text += target_texts[i]
            i += 1
        p = prompt + text
        #print(f"Prompt ({len(p)}):", p)
        # print(i, len(p))

        # try the API call and if it fails, wait 10 seconds and retry again (max 3 times)
        for j in range(6):
            try:
                response = openai.ChatCompletion.create(
                    model="gpt-3.5-turbo",
                    messages=[{"role": "user", "content": p}]
                )
                break
            except:
                print("error calling API, retrying...")
                time.sleep(10)

        r = response["choices"][0]["message"]["content"]
        rez.append(r)
        # convert r to int
        try:
            b = int(r[:1])
        except:
            b = -1
            print("error parsing"+r)
        print(i, len(p), "-", r, b, end=": ")
        # if i % 10 print
        if i % 10 == 0:
            print()
        rez_keys.append(b)
    return rez,rez_keys

# testing Climate Detection on ChatGPT
## Classification classes:
### 0 - is not climate change-related
### 1 - is climate change-related

In [ ]:
!pip install datasets
from datasets import load_dataset

dataset = load_dataset("climatebert/climate_detection")

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


  0%|          | 0/2 [00:00<?, ?it/s]

In [ ]:
dataset

DatasetDict({
    train: Dataset({
        features: ['text', 'label'],
        num_rows: 1300
    })
    test: Dataset({
        features: ['text', 'label'],
        num_rows: 400
    })
})

In [ ]:
dataset['test']

Dataset({
    features: ['text', 'label'],
    num_rows: 400
})

In [ ]:
data = []
for i in range(0,len(dataset['test']['text'])):
  data.append([dataset['test']['text'][i],dataset['test']['label'][i]])

print(data)

[['Sustainable strategy ‘red lines’ For our sustainable strategy range, we incorporate a series of proprietary ‘red lines’ in order to ensure the poorest- performing companies from an ESG perspective are not eligible for investment.', 1], ['Verizon’s environmental, health and safety management system provides a framework for identifying, controlling, and reducing the risks associated with the environments in which we operate. Besides regular management system assessments, internal and third-party compliance audits and inspections are performed annually at hundreds of facilities worldwide. The goal of these assessments is to identify and correct site-specific issues, and to educate and empower facility managers and supervisors to implement corrective actions. Verizon’s environment, health and safety efforts are directed and supported by experienced experts around the world that support our operations and facilities.', 1], ['In 2019, the Company closed a series of transactions related to

In [ ]:
import pandas as pd
df = pd.DataFrame(data=data,columns=["text","label"])

In [ ]:
df

,text,label
0,Sustainable strategy ‘red lines’ For our susta...,1
1,"Verizon’s environmental, health and safety man...",1
2,"In 2019, the Company closed a series of transa...",1
3,"In December 2020, the AUC approved the Electri...",1
4,"Finally, there is a reputational risk linked t...",1
...,...,...
395,"In 2020, Banco do Brasil Foundation celebrated...",1
396,Climate change is producing changes in weather...,1
397,A sound and certain regulatory and fiscal envi...,0
398,"Across our global workforce, 20% of Gold Field...",0


In [ ]:
prompt = 'You are the sustainability, environment, and climate change expert. Is the following text about sustainability, the environment, or climate change? Answer only with 1 if the text is sustainability, environment or climate change related or 0 if not: \n\n'
texts = df["text"].to_list()
rez, rez_keys = batch_gpt_len(prompt, texts, 1)

1 480 - 1 (sustainability related) 1: 2 938 - 1 (environment and sustainability) 1: 3 867 - 0 0: 4 1180 - 0 0: 5 670 - 1 (climate change related) 1: 6 639 - 1 - Sustainability and environment related. 1: 7 636 - 1 (climate change related) 1: 8 684 - 1 (sustainability) 1: 9 780 - 1 (climate change) 1: 10 555 - 1 (climate change) 1: 
11 844 - 0 0: 12 511 - 1 (climate change related) 1: error parsingClimate change (1)
13 523 - Climate change (1) -1: 14 757 - 0 0: 15 477 - 1 (Environment) 1: 16 686 - 0 0: 17 536 - 0 0: 18 697 - 1 (climate change) 1: error calling API, retrying...
19 620 - 1 (climate change) 1: 20 451 - 1 (climate change related) 1: 
21 927 - 1 (environment related) 1: 22 686 - 0 0: error parsingclimate change 1
23 962 - climate change 1 -1: 24 907 - 1 (climate change) 1: 25 862 - 1 (climate change related) 1: 26 613 - 1 (climate change) 1: 27 562 - 1 (sustainability related) 1: 28 502 - 1 (Climate change related) 1: 29 733 - 1 (climate change) 1: 30 865 - 1 (climate change

In [ ]:
rez

['1 (sustainability related)',
 '1 (environment and sustainability)',
 '0',
 '0',
 '1 (climate change related)',
 '1 - Sustainability and environment related.',
 '1 (climate change related)',
 '1 (sustainability)',
 '1 (climate change)',
 '1 (climate change)',
 '0',
 '1 (climate change related)',
 'Climate change (1)',
 '0',
 '1 (Environment)',
 '0',
 '0',
 '1 (climate change)',
 '1 (climate change)',
 '1 (climate change related)',
 '1 (environment related)',
 '0',
 'climate change 1',
 '1 (climate change)',
 '1 (climate change related)',
 '1 (climate change)',
 '1 (sustainability related)',
 '1 (Climate change related)',
 '1 (climate change)',
 '1 (climate change related)',
 '1 (environment and climate change related)',
 '1 (climate change related)',
 '1 (sustainability)',
 'climate change - 1',
 '1',
 '1 (sustainability and environment related)',
 '1 (sustainability and environment related)',
 '1 (environment and sustainability related)',
 '1 (environmental and social issues, environ

In [ ]:
rez_keys

[1,
 1,
 0,
 0,
 1,
 1,
 1,
 1,
 1,
 1,
 0,
 1,
 -1,
 0,
 1,
 0,
 0,
 1,
 1,
 1,
 1,
 0,
 -1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 -1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 0,
 1,
 1,
 1,
 1,
 1,
 1,
 0,
 1,
 0,
 -1,
 -1,
 1,
 0,
 -1,
 1,
 1,
 0,
 1,
 1,
 1,
 1,
 0,
 1,
 1,
 1,
 0,
 0,
 1,
 -1,
 0,
 1,
 1,
 0,
 1,
 0,
 1,
 1,
 1,
 -1,
 1,
 0,
 1,
 -1,
 1,
 -1,
 1,
 -1,
 1,
 1,
 -1,
 1,
 1,
 0,
 0,
 1,
 1,
 0,
 1,
 1,
 1,
 0,
 0,
 0,
 1,
 1,
 1,
 1,
 0,
 0,
 0,
 1,
 1,
 0,
 0,
 1,
 0,
 1,
 0,
 1,
 0,
 0,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 -1,
 1,
 0,
 1,
 -1,
 1,
 1,
 0,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 -1,
 1,
 1,
 1,
 1,
 1,
 -1,
 -1,
 1,
 0,
 1,
 1,
 1,
 0,
 -1,
 0,
 1,
 0,
 1,
 0,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 -1,
 0,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 0,
 1,
 0,
 0,
 1,
 0,
 1,
 0,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 0,
 1,
 1,
 0,
 1,
 0,
 1,
 0,
 0,
 1,
 1,
 1,
 1,
 0,
 1,
 0,
 1,
 0,
 1,
 1,
 0,
 1,
 0,
 1,
 0,
 1,
 1,
 1,
 1,
 1,
 1,
 

In [ ]:
df['gpt-label'] = rez_keys

In [ ]:
df['gpt-explanations'] = rez

In [ ]:
df

,text,label,gpt-label,gpt-explanations
0,Sustainable strategy ‘red lines’ For our susta...,1,1,1 (sustainability related)
1,"Verizon’s environmental, health and safety man...",1,1,1 (environment and sustainability)
2,"In 2019, the Company closed a series of transa...",1,0,0
3,"In December 2020, the AUC approved the Electri...",1,0,0
4,"Finally, there is a reputational risk linked t...",1,1,1 (climate change related)
...,...,...,...,...
395,"In 2020, Banco do Brasil Foundation celebrated...",1,1,1 (sustainability related)
396,Climate change is producing changes in weather...,1,-1,climate change: 1
397,A sound and certain regulatory and fiscal envi...,0,0,0
398,"Across our global workforce, 20% of Gold Field...",0,0,0


In [ ]:
df['gpt-label'].value_counts()

 1    259
 0    112
-1     29
Name: gpt-label, dtype: int64

In [ ]:
df[df['gpt-label']==-1]

,text,label,gpt-label,gpt-explanations
12,Note: 1. Please refer to page 29 within Metric...,1,-1,Climate change (1)
22, To exclude the least energy efficient hydroc...,1,-1,climate change 1
33,"Over the last few years, we have witnessed a c...",1,-1,climate change - 1
59,Our aim in 2021 is to be able to report the we...,1,-1,climate change - 1
60,Logistics remains focused on driving freight t...,1,-1,Climate change: 1
63,Managing weather impacts Tropical cyclones (hu...,1,-1,Climate change (1)
78,"If we are unable to find economically viable, ...",1,-1,Climate change. (1)
88,Other disruptions in the manufacturing process...,1,-1,Environment: 1 (refers to natural disasters an...
92,"Traffic infrastructures (roads, airports and r...",1,-1,Environment: 1
94,Climate change is a challenge faced by the ent...,1,-1,Climate change: 1


In [ ]:
df['gpt-label'] = df['gpt-label'].replace(-1,1)

In [ ]:
df['gpt-label'].value_counts()

1    288
0    112
Name: gpt-label, dtype: int64

In [ ]:
df.to_csv("/content/drive/MyDrive/DS-Environment-Project/ChatGPT Results/chatgpt_climate_detection.csv",index=False)

In [ ]:
# calculate the precision and f1 score for df columns label and prediction
from sklearn.metrics import precision_recall_fscore_support,f1_score
sent_col = 'gpt-label'
print(precision_recall_fscore_support(df['label'], df[sent_col], average='macro'))

# f1 score only
print(f1_score(df['label'], df[sent_col], average='macro'))

# confusion matrix
from sklearn.metrics import confusion_matrix
print(confusion_matrix(df['label'], df[sent_col]))

# performnce report

from sklearn.metrics import classification_report
print(classification_report(df['label'], df[sent_col]))

(0.7889384920634921, 0.8640625, 0.814967105263158, None)
0.814967105263158
[[ 69  11]
 [ 43 277]]
              precision    recall  f1-score   support

           0       0.62      0.86      0.72        80
           1       0.96      0.87      0.91       320

    accuracy                           0.86       400
   macro avg       0.79      0.86      0.81       400
weighted avg       0.89      0.86      0.87       400



# testing Climate Sentiment on ChatGPT
## Sentiment classes:
### 0 - risk
### 1 - neutral
### 2 - opportunity

In [ ]:
!pip install datasets
from datasets import load_dataset

dataset = load_dataset("climatebert/climate_sentiment")

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


Extracting data files:   0%|          | 0/2 [00:00<?, ?it/s]

Generating train split:   0%|          | 0/1000 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/320 [00:00<?, ? examples/s]

Dataset parquet downloaded and prepared to /root/.cache/huggingface/datasets/climatebert___parquet/climatebert--climate_sentiment-cde94fbdb0f4004a/0.0.0/2a3b91fbd88a2c90d1dbbb32b460cf621d31bd5b05b934492fdef7d8d6f236ec. Subsequent calls will reuse this data.


  0%|          | 0/2 [00:00<?, ?it/s]

In [ ]:
dataset

DatasetDict({
    train: Dataset({
        features: ['text', 'label'],
        num_rows: 1000
    })
    test: Dataset({
        features: ['text', 'label'],
        num_rows: 320
    })
})

In [ ]:
dataset['test']

Dataset({
    features: ['text', 'label'],
    num_rows: 320
})

In [ ]:
data = []
for i in range(0,len(dataset['test']['text'])):
  data.append([dataset['test']['text'][i],dataset['test']['label'][i]])

print(data)

[['Sustainable strategy ‘red lines’ For our sustainable strategy range, we incorporate a series of proprietary ‘red lines’ in order to ensure the poorest- performing companies from an ESG perspective are not eligible for investment.', 0], ['Verizon’s environmental, health and safety management system provides a framework for identifying, controlling, and reducing the risks associated with the environments in which we operate. Besides regular management system assessments, internal and third-party compliance audits and inspections are performed annually at hundreds of facilities worldwide. The goal of these assessments is to identify and correct site-specific issues, and to educate and empower facility managers and supervisors to implement corrective actions. Verizon’s environment, health and safety efforts are directed and supported by experienced experts around the world that support our operations and facilities.', 1], ['In 2019, the Company closed a series of transactions related to

In [ ]:
import pandas as pd
df = pd.DataFrame(data=data,columns=["text","label"])

In [ ]:
df

,text,label
0,Sustainable strategy ‘red lines’ For our susta...,0
1,"Verizon’s environmental, health and safety man...",1
2,"In 2019, the Company closed a series of transa...",1
3,"In December 2020, the AUC approved the Electri...",0
4,"Finally, there is a reputational risk linked t...",0
...,...,...
315,Indirect emissions result from operational act...,1
316,"All data in this TCFD report is as of, or for ...",1
317,Outcome: The bank explained that it would be w...,1
318,"In 2020, Banco do Brasil Foundation celebrated...",2


In [ ]:
prompt = 'You are the sustainability, environment, and climate change expert. Does the following text indicate risk, is neutral or indicates an opportunity about sustainability, the environment, or climate change? Answer only with 0 if the text indicates risk, answer only with 1 if the text is neutral or with 2 if the text indicates an opportunity: \n\n'
texts = df["text"].to_list()
rez, rez_keys = batch_gpt_len(prompt, texts, 1)

1 572 - 2 2: 2 1030 - 2 2: 3 959 - 2 (indicates an opportunity as the Company is moving away from fossil fuel-based electricity generation business) 2: 4 1272 - 1 (neutral) 1: 5 762 - 2 (indicates an opportunity) 2: 6 731 - 2 2: 7 728 - 0 0: 8 872 - 2 (indicates an opportunity) 2: 9 647 - 0 0: 10 603 - 2 2: 
11 615 - 1 (neutral) 1: 12 569 - 2 2: 13 628 - 2 2: 14 789 - 1 1: error calling API, retrying...
15 712 - 0 0: 16 543 - 1 1: 17 1019 - 1 1: 18 778 - 2 (indicates opportunity, as the company is providing aid to employees affected by natural disasters or emergencies) 2: 19 1054 - 2 2: 20 999 - 1 (neutral) 1: 
21 954 - 0 - indicates risk. 0: 22 705 - 0 0: 23 654 - 2 2: 24 594 - 2 2: 25 825 - 2 (indicates an opportunity) 2: 26 957 - 0 (indicates risk) 0: 27 955 - 2 2: 28 552 - 2 2: 29 651 - 2 2: 30 846 - 0 (indicates risk) 0: 
31 1010 - 2 2: 32 1180 - 2 2: 33 935 - 2 2: 34 677 - 2 2: 35 850 - 0 0: 36 667 - 2 2: 37 767 - 0 (indicates risk) 0: 38 645 - 2 2: 39 922 - 2 2: 40 801 - 1 1: 
4

In [ ]:
rez

['2',
 '2',
 '2 (indicates an opportunity as the Company is moving away from fossil fuel-based electricity generation business)',
 '1 (neutral)',
 '2 (indicates an opportunity)',
 '2',
 '0',
 '2 (indicates an opportunity)',
 '0',
 '2',
 '1 (neutral)',
 '2',
 '2',
 '1',
 '0',
 '1',
 '1',
 '2 (indicates opportunity, as the company is providing aid to employees affected by natural disasters or emergencies)',
 '2',
 '1 (neutral)',
 '0 - indicates risk.',
 '0',
 '2',
 '2',
 '2 (indicates an opportunity)',
 '0 (indicates risk)',
 '2',
 '2',
 '2',
 '0 (indicates risk)',
 '2',
 '2',
 '2',
 '2',
 '0',
 '2',
 '0 (indicates risk)',
 '2',
 '2',
 '1',
 '2 (indicates an opportunity)',
 '1',
 '2',
 '2',
 '1',
 '2',
 '1',
 '2',
 '0 (indicates risk)',
 '2',
 '2',
 '0 (indicates risk)',
 '2',
 '1 (neutral)',
 '2',
 '2 (indicates an opportunity)',
 '2 (indicates opportunity)',
 '0 (risk)',
 '0',
 '1',
 '2',
 '2',
 '1',
 '1',
 '0',
 '2',
 '0',
 '1',
 '2',
 '0',
 '0 (indicates risk)',
 '2',
 '1',
 '2 (indi

In [ ]:
rez_keys

[2,
 2,
 2,
 1,
 2,
 2,
 0,
 2,
 0,
 2,
 1,
 2,
 2,
 1,
 0,
 1,
 1,
 2,
 2,
 1,
 0,
 0,
 2,
 2,
 2,
 0,
 2,
 2,
 2,
 0,
 2,
 2,
 2,
 2,
 0,
 2,
 0,
 2,
 2,
 1,
 2,
 1,
 2,
 2,
 1,
 2,
 1,
 2,
 0,
 2,
 2,
 0,
 2,
 1,
 2,
 2,
 2,
 0,
 0,
 1,
 2,
 2,
 1,
 1,
 0,
 2,
 0,
 1,
 2,
 0,
 0,
 2,
 1,
 2,
 2,
 2,
 0,
 2,
 2,
 2,
 0,
 0,
 0,
 1,
 2,
 2,
 2,
 1,
 2,
 1,
 2,
 2,
 0,
 1,
 2,
 2,
 1,
 0,
 2,
 1,
 2,
 2,
 1,
 1,
 2,
 2,
 1,
 2,
 2,
 0,
 2,
 2,
 2,
 2,
 2,
 2,
 0,
 1,
 2,
 2,
 0,
 2,
 1,
 2,
 2,
 2,
 1,
 2,
 0,
 0,
 0,
 2,
 2,
 2,
 2,
 2,
 2,
 1,
 2,
 2,
 1,
 2,
 2,
 2,
 0,
 2,
 1,
 2,
 2,
 2,
 2,
 2,
 0,
 0,
 1,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 1,
 1,
 1,
 2,
 2,
 2,
 0,
 2,
 2,
 2,
 0,
 1,
 0,
 0,
 0,
 1,
 1,
 2,
 2,
 1,
 1,
 0,
 2,
 1,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 1,
 2,
 2,
 2,
 0,
 2,
 2,
 1,
 0,
 2,
 2,
 2,
 -1,
 1,
 0,
 2,
 2,
 2,
 0,
 2,
 0,
 1,
 0,
 2,
 2,
 1,
 2,
 2,
 2,
 0,
 1,
 2,
 2,
 0,
 1,
 2,
 2,
 2,
 2,
 2,
 0,
 1,
 2,
 2,
 1,
 2,
 2,
 0,
 0,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 -1

In [ ]:
df['gpt-label'] = rez_keys

In [ ]:
df['gpt-explanations'] = rez

In [ ]:
df

,text,label,gpt-label,gpt-explanations
0,Sustainable strategy ‘red lines’ For our susta...,0,2,2
1,"Verizon’s environmental, health and safety man...",1,2,2
2,"In 2019, the Company closed a series of transa...",1,2,2 (indicates an opportunity as the Company is ...
3,"In December 2020, the AUC approved the Electri...",0,1,1 (neutral)
4,"Finally, there is a reputational risk linked t...",0,2,2 (indicates an opportunity)
...,...,...,...,...
315,Indirect emissions result from operational act...,1,1,1
316,"All data in this TCFD report is as of, or for ...",1,1,1
317,Outcome: The bank explained that it would be w...,1,2,2 (indicates an opportunity)
318,"In 2020, Banco do Brasil Foundation celebrated...",2,2,2


In [ ]:
df['gpt-label'].value_counts()

 2    193
 1     64
 0     59
-1      4
Name: gpt-label, dtype: int64

In [ ]:
df[df['gpt-label']==-1]

,text,label,gpt-label,gpt-explanations
205,We also adopted specific conditions for granti...,1,-1,of sustainable development.\n\n2
249,Although climate change has not yet had a mate...,0,-1,"business strategy and operations, and is commi..."
268,As we invest in Manufactured Capital to improv...,2,-1,and Human Capital. \n\n2 (indicates an opportu...
270,"Increasing population, urbanisation, our own i...",0,-1,"Technology, we can increase our resource manag..."


In [ ]:
df['gpt-label'] = df['gpt-label'].replace(-1,2)

In [ ]:
df['gpt-label'].value_counts()

2    197
1     64
0     59
Name: gpt-label, dtype: int64

In [ ]:
df.to_csv("/content/drive/MyDrive/DS-Environment-Project/ChatGPT Results/chatgpt_climate_sentiment.csv",index=False)

In [ ]:
# calculate the precision and f1 score for df columns label and prediction
from sklearn.metrics import precision_recall_fscore_support,f1_score
sent_col = 'gpt-label'
print(precision_recall_fscore_support(df['label'], df[sent_col], average='macro'))

# f1 score only
print(f1_score(df['label'], df[sent_col], average='macro'))

# confusion matrix
from sklearn.metrics import confusion_matrix
print(confusion_matrix(df['label'], df[sent_col]))

# performnce report

from sklearn.metrics import classification_report
print(classification_report(df['label'], df[sent_col]))

(0.6405698829906221, 0.5969675442040844, 0.49576667628394216, None)
0.49576667628394216
[[ 56  17  33]
 [  3  46 114]
 [  0   1  50]]
              precision    recall  f1-score   support

           0       0.95      0.53      0.68       106
           1       0.72      0.28      0.41       163
           2       0.25      0.98      0.40        51

    accuracy                           0.48       320
   macro avg       0.64      0.60      0.50       320
weighted avg       0.72      0.47      0.50       320



# testing Climate Classification on commitments and actions on ChatGPT
## Classification classes:
### 0 - paragraph is not about climate commitments and actions
### 1 - paragraph is about climate commitments and actions


In [ ]:
!pip install datasets
from datasets import load_dataset

dataset = load_dataset("climatebert/climate_commitments_actions")

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


Extracting data files:   0%|          | 0/2 [00:00<?, ?it/s]

Generating train split:   0%|          | 0/1000 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/320 [00:00<?, ? examples/s]

Dataset parquet downloaded and prepared to /root/.cache/huggingface/datasets/climatebert___parquet/climatebert--climate_commitments_actions-c39067b6628a5441/0.0.0/2a3b91fbd88a2c90d1dbbb32b460cf621d31bd5b05b934492fdef7d8d6f236ec. Subsequent calls will reuse this data.


  0%|          | 0/2 [00:00<?, ?it/s]

In [ ]:
dataset

DatasetDict({
    train: Dataset({
        features: ['text', 'label'],
        num_rows: 1000
    })
    test: Dataset({
        features: ['text', 'label'],
        num_rows: 320
    })
})

In [ ]:
dataset['test']

Dataset({
    features: ['text', 'label'],
    num_rows: 320
})

In [ ]:
data = []
for i in range(0,len(dataset['test']['text'])):
  data.append([dataset['test']['text'][i],dataset['test']['label'][i]])

print(data)

[['Sustainable strategy ‘red lines’ For our sustainable strategy range, we incorporate a series of proprietary ‘red lines’ in order to ensure the poorest- performing companies from an ESG perspective are not eligible for investment.', 1], ['Verizon’s environmental, health and safety management system provides a framework for identifying, controlling, and reducing the risks associated with the environments in which we operate. Besides regular management system assessments, internal and third-party compliance audits and inspections are performed annually at hundreds of facilities worldwide. The goal of these assessments is to identify and correct site-specific issues, and to educate and empower facility managers and supervisors to implement corrective actions. Verizon’s environment, health and safety efforts are directed and supported by experienced experts around the world that support our operations and facilities.', 0], ['In 2019, the Company closed a series of transactions related to

In [ ]:
import pandas as pd
df = pd.DataFrame(data=data,columns=["text","label"])

In [ ]:
df

,text,label
0,Sustainable strategy ‘red lines’ For our susta...,1
1,"Verizon’s environmental, health and safety man...",0
2,"In 2019, the Company closed a series of transa...",1
3,"In December 2020, the AUC approved the Electri...",0
4,"Finally, there is a reputational risk linked t...",0
...,...,...
315,Indirect emissions result from operational act...,0
316,"All data in this TCFD report is as of, or for ...",0
317,Outcome: The bank explained that it would be w...,1
318,"In 2020, Banco do Brasil Foundation celebrated...",1


In [ ]:
prompt = 'You are the sustainability, environment, and climate change expert. Is the following text about climate commitments and actions or not? Answer only with 0 if the text is not about climate commitments and actions and with 1 if the text is about climate commitments and actions: \n\n'
texts = df["text"].to_list()
rez, rez_keys = batch_gpt_len(prompt, texts, 1)

1 508 - 0 0: 2 966 - 0 0: 3 895 - 0 0: 4 1208 - 0 0: 5 698 - 0 0: 6 667 - 1 1: 7 664 - 0 0: 8 808 - 1 1: 9 583 - 1 1: 10 539 - 1 1: 
11 551 - 1 1: 12 505 - 0 0: 13 564 - 0 0: error calling API, retrying...
14 725 - 1 1: 15 648 - 1 1: 16 479 - 1 1: 17 955 - 0 0: 18 714 - 0 0: 19 990 - 1 (This text is about climate commitments and actions, specifically about excluding certain hydrocarbons and committing to offsetting the company's direct carbon footprint.) 1: 20 935 - 1 1: 
21 890 - 0 0: 22 641 - 0 0: 23 590 - 0 0: 24 530 - 1 1: 25 761 - 1 1: 26 893 - 1 1: 27 891 - 0 0: 28 488 - 1 1: 29 587 - 0 0: 30 782 - 1 1: 
31 946 - 0 0: 32 1116 - 1 1: 33 871 - 1 1: 34 613 - 1 1: 35 786 - 0 0: 36 603 - 1 1: 37 703 - 1 1: 38 581 - 1 1: 39 858 - 1 1: 40 737 - 1 1: 
41 529 - 1 1: 42 576 - 1 1: 43 836 - 1 1: 44 833 - 0 0: 45 547 - 0 0: 46 508 - 1 1: 47 497 - 1 1: 48 734 - 1 1: 49 603 - 1 1: 50 1426 - 1 (about climate commitments and actions) 1: 
51 737 - 1 1: 52 701 - 0 0: 53 513 - 1 1: 54 632 - 0 0: er

In [ ]:
rez

['0',
 '0',
 '0',
 '0',
 '0',
 '1',
 '0',
 '1',
 '1',
 '1',
 '1',
 '0',
 '0',
 '1',
 '1',
 '1',
 '0',
 '0',
 "1 (This text is about climate commitments and actions, specifically about excluding certain hydrocarbons and committing to offsetting the company's direct carbon footprint.)",
 '1',
 '0',
 '0',
 '0',
 '1',
 '1',
 '1',
 '0',
 '1',
 '0',
 '1',
 '0',
 '1',
 '1',
 '1',
 '0',
 '1',
 '1',
 '1',
 '1',
 '1',
 '1',
 '1',
 '1',
 '0',
 '0',
 '1',
 '1',
 '1',
 '1',
 '1 (about climate commitments and actions)',
 '1',
 '0',
 '1',
 '0',
 '1',
 '0',
 '1',
 '0',
 '0',
 '1',
 '0',
 '1',
 '0',
 '0',
 '1',
 '1',
 '1',
 '1',
 '1',
 '1.',
 '0',
 '1',
 '1',
 '1',
 '1',
 '1',
 '0',
 '0',
 '1',
 '0',
 '1',
 '1',
 '0',
 '1',
 '1',
 '0',
 '1.',
 '0',
 '0',
 '0',
 '1',
 '0',
 '0',
 '0',
 '1',
 '1',
 '0',
 '0',
 '1',
 '0',
 '1',
 '1',
 '0',
 '0',
 '1',
 '0',
 '1',
 '1',
 '1',
 '0',
 '0',
 '1',
 '1',
 '1',
 '1',
 '0',
 '1',
 '1',
 '0',
 '1',
 '1',
 '1',
 '0',
 '1',
 '1',
 '1',
 '0',
 '1',
 '0',
 '0',
 '0',


In [ ]:
rez_keys

[0,
 0,
 0,
 0,
 0,
 1,
 0,
 1,
 1,
 1,
 1,
 0,
 0,
 1,
 1,
 1,
 0,
 0,
 1,
 1,
 0,
 0,
 0,
 1,
 1,
 1,
 0,
 1,
 0,
 1,
 0,
 1,
 1,
 1,
 0,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 0,
 0,
 1,
 1,
 1,
 1,
 1,
 1,
 0,
 1,
 0,
 1,
 0,
 1,
 0,
 0,
 1,
 0,
 1,
 0,
 0,
 1,
 1,
 1,
 1,
 1,
 1,
 0,
 1,
 1,
 1,
 1,
 1,
 0,
 0,
 1,
 0,
 1,
 1,
 0,
 1,
 1,
 0,
 1,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 1,
 1,
 0,
 0,
 1,
 0,
 1,
 1,
 0,
 0,
 1,
 0,
 1,
 1,
 1,
 0,
 0,
 1,
 1,
 1,
 1,
 0,
 1,
 1,
 0,
 1,
 1,
 1,
 0,
 1,
 1,
 1,
 0,
 1,
 0,
 0,
 0,
 1,
 0,
 1,
 1,
 1,
 0,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 0,
 0,
 0,
 0,
 1,
 1,
 1,
 1,
 0,
 0,
 0,
 1,
 1,
 1,
 1,
 0,
 1,
 1,
 0,
 1,
 0,
 1,
 1,
 0,
 1,
 1,
 1,
 1,
 0,
 0,
 0,
 0,
 1,
 1,
 0,
 1,
 1,
 1,
 1,
 0,
 1,
 0,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 0,
 1,
 1,
 0,
 1,
 1,
 0,
 0,
 1,
 1,
 0,
 1,
 1,
 0,
 1,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 1,
 1,
 1,
 1,
 1,
 0,
 0,
 1,
 1,
 1,
 0,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 0,
 1,
 1,
 0,
 1,
 0,
 1,
 1,
 1,
 1,
 1,
 1,
 -1,

In [ ]:
df['gpt-label'] = rez_keys

In [ ]:
df['gpt-explanations'] = rez

In [ ]:
df

,text,label,gpt-label,gpt-explanations
0,Sustainable strategy ‘red lines’ For our susta...,1,0,0
1,"Verizon’s environmental, health and safety man...",0,0,0
2,"In 2019, the Company closed a series of transa...",1,0,0
3,"In December 2020, the AUC approved the Electri...",0,0,0
4,"Finally, there is a reputational risk linked t...",0,0,0
...,...,...,...,...
315,Indirect emissions result from operational act...,0,1,1
316,"All data in this TCFD report is as of, or for ...",0,0,0
317,Outcome: The bank explained that it would be w...,1,1,1
318,"In 2020, Banco do Brasil Foundation celebrated...",1,0,0


In [ ]:
df['gpt-label'].value_counts()

 1    194
 0    123
-1      3
Name: gpt-label, dtype: int64

In [ ]:
df[df['gpt-label']==-1]

,text,label,gpt-label,gpt-explanations
249,Although climate change has not yet had a mate...,0,-1,business operations and is committed to addres...
259,C u s t o m e r s ’ p r e f e r e n c e f o r ...,0,-1,commitments.\n\n1
268,As we invest in Manufactured Capital to improv...,1,-1,and Human Capital. \n\n0


In [ ]:
df['gpt-label'] = df['gpt-label'].replace(-1,1)

In [ ]:
df['gpt-label'].value_counts()

1    197
0    123
Name: gpt-label, dtype: int64

In [ ]:
df.to_csv("/content/drive/MyDrive/DS-Environment-Project/ChatGPT Results/chatgpt_climate_commitments_and_actions.csv",index=False)

In [ ]:
# calculate the precision and f1 score for df columns label and prediction
from sklearn.metrics import precision_recall_fscore_support,f1_score
sent_col = 'gpt-label'
print(precision_recall_fscore_support(df['label'], df[sent_col], average='macro'))

# f1 score only
print(f1_score(df['label'], df[sent_col], average='macro'))

# confusion matrix
from sklearn.metrics import confusion_matrix
print(confusion_matrix(df['label'], df[sent_col]))

# performnce report

from sklearn.metrics import classification_report
print(classification_report(df['label'], df[sent_col]))

(0.5770500598406999, 0.5858154072439787, 0.5252272169000245, None)
0.5252272169000245
[[ 97 125]
 [ 26  72]]
              precision    recall  f1-score   support

           0       0.79      0.44      0.56       222
           1       0.37      0.73      0.49        98

    accuracy                           0.53       320
   macro avg       0.58      0.59      0.53       320
weighted avg       0.66      0.53      0.54       320



# testing Climate Specificity Classification on ChatGPT
## Classification classes:
### 0 - paragraph is not specific
### 1 - paragraph is specific


In [ ]:
!pip install datasets
from datasets import load_dataset

dataset = load_dataset("climatebert/climate_specificity")

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


  0%|          | 0/2 [00:00<?, ?it/s]

In [ ]:
dataset

DatasetDict({
    train: Dataset({
        features: ['text', 'label'],
        num_rows: 1000
    })
    test: Dataset({
        features: ['text', 'label'],
        num_rows: 320
    })
})

In [ ]:
dataset['test']

Dataset({
    features: ['text', 'label'],
    num_rows: 320
})

In [ ]:
data = []
for i in range(0,len(dataset['test']['text'])):
  data.append([dataset['test']['text'][i],dataset['test']['label'][i]])

print(data)

[['Sustainable strategy ‘red lines’ For our sustainable strategy range, we incorporate a series of proprietary ‘red lines’ in order to ensure the poorest- performing companies from an ESG perspective are not eligible for investment.', 0], ['Verizon’s environmental, health and safety management system provides a framework for identifying, controlling, and reducing the risks associated with the environments in which we operate. Besides regular management system assessments, internal and third-party compliance audits and inspections are performed annually at hundreds of facilities worldwide. The goal of these assessments is to identify and correct site-specific issues, and to educate and empower facility managers and supervisors to implement corrective actions. Verizon’s environment, health and safety efforts are directed and supported by experienced experts around the world that support our operations and facilities.', 0], ['In 2019, the Company closed a series of transactions related to

In [ ]:
import pandas as pd
df = pd.DataFrame(data=data,columns=["text","label"])

In [ ]:
df

,text,label
0,Sustainable strategy ‘red lines’ For our susta...,0
1,"Verizon’s environmental, health and safety man...",0
2,"In 2019, the Company closed a series of transa...",1
3,"In December 2020, the AUC approved the Electri...",1
4,"Finally, there is a reputational risk linked t...",0
...,...,...
315,Indirect emissions result from operational act...,0
316,"All data in this TCFD report is as of, or for ...",0
317,Outcome: The bank explained that it would be w...,1
318,"In 2020, Banco do Brasil Foundation celebrated...",0


In [ ]:
prompt = 'You are the sustainability, environment, and climate change expert. Is the following text specific about sustainability, environment, and climate change or not? Answer only with 0 if the text is not not specific and with 1 if the text is specific: \n\n'
texts = df["text"].to_list()
rez, rez_keys = batch_gpt_len(prompt, texts, 1)

1 479 - 1 1: 2 937 - 1 1: 3 866 - 0 0: 4 1179 - 0 0: 5 669 - 1 1: 6 638 - 1 1: 7 635 - 1 1: 8 779 - 1 1: 9 554 - 1 1: 10 510 - 1 1: 
11 522 - 1 (specific) 1: 12 476 - 1 1: error calling API, retrying...
13 535 - 1 1: 14 696 - 1 1: 15 619 - 1 1: 16 450 - 1 1: error calling API, retrying...
17 926 - 0 0: 18 685 - 0 0: error parsingclimate protection projects certified by independent third parties. 

1 (specific)
19 961 - climate protection projects certified by independent third parties. 

1 (specific) -1: 20 906 - 1 1: 
21 861 - 1 (specific) 1: 22 612 - 1 1: 23 561 - 1 1: 24 501 - 1 1: 25 732 - 1 1: 26 864 - 1 1: 27 862 - 0 0: 28 459 - 1 1: 29 558 - 1 1: 30 753 - 1 1: 
31 917 - 1 1: 32 1087 - 1 (specific about sustainability and environment) 1: 33 842 - 1 1: error calling API, retrying...
34 584 - 1 1: 35 757 - 1 1: 36 574 - 1 1: 37 674 - 1 1: error calling API, retrying...
38 552 - 1 (specific) 1: 39 829 - 1 1: 40 708 - 1 1: 
41 500 - 1 1: 42 547 - 1 1: 43 807 - 1 1: 44 804 - 1 1: 45 5

In [ ]:
rez

['1',
 '1',
 '0',
 '0',
 '1',
 '1',
 '1',
 '1',
 '1',
 '1',
 '1 (specific)',
 '1',
 '1',
 '1',
 '1',
 '1',
 '0',
 '0',
 'climate protection projects certified by independent third parties. \n\n1 (specific)',
 '1',
 '1 (specific)',
 '1',
 '1',
 '1',
 '1',
 '1',
 '0',
 '1',
 '1',
 '1',
 '1',
 '1 (specific about sustainability and environment)',
 '1',
 '1',
 '1',
 '1',
 '1',
 '1 (specific)',
 '1',
 '1',
 '1',
 '1',
 '1',
 '1',
 '0',
 '1',
 '1',
 '1',
 '1',
 '1',
 '1',
 '0',
 '1',
 '0',
 '1',
 '1',
 '1',
 '0',
 '1',
 '1 (specific)',
 '1',
 '1',
 '1',
 '1',
 '1',
 '1',
 '1',
 '1',
 '1',
 '1',
 '1',
 '1',
 '1',
 '1',
 '1',
 '1',
 '0',
 '0 (not specific to sustainability, environment, and climate change)',
 '1',
 '1',
 '1',
 '1',
 '1',
 '1',
 '1',
 '1',
 '1',
 '1',
 '1',
 '0 (not specific)',
 '1',
 '1',
 '0',
 '0',
 '1',
 '1',
 '0',
 '0',
 '1 (specific)',
 '1',
 '1',
 '1',
 '0 (not specific)',
 '0',
 '1',
 '1',
 '1',
 '1',
 '1',
 '1',
 '1',
 '1',
 '1',
 '1',
 '1',
 '1.',
 '1',
 '1',
 '1',
 '1

In [ ]:
rez_keys

[1,
 1,
 0,
 0,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 0,
 0,
 -1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 0,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 0,
 1,
 1,
 1,
 1,
 1,
 1,
 0,
 1,
 0,
 1,
 1,
 1,
 0,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 0,
 0,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 0,
 1,
 1,
 0,
 0,
 1,
 1,
 0,
 0,
 1,
 1,
 1,
 1,
 0,
 0,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 0,
 1,
 0,
 0,
 1,
 1,
 1,
 1,
 1,
 1,
 0,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 0,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 0,
 0,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 0,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 0,
 0,
 1,
 0,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 0,
 0,
 1,
 1,
 1,
 0,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 0,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 -1

In [ ]:
df['gpt-label'] = rez_keys

In [ ]:
df['gpt-explanations'] = rez

In [ ]:
df

,text,label,gpt-label,gpt-explanations
0,Sustainable strategy ‘red lines’ For our susta...,0,1,1
1,"Verizon’s environmental, health and safety man...",0,1,1
2,"In 2019, the Company closed a series of transa...",1,0,0
3,"In December 2020, the AUC approved the Electri...",1,0,0
4,"Finally, there is a reputational risk linked t...",0,1,1
...,...,...,...,...
315,Indirect emissions result from operational act...,0,1,1
316,"All data in this TCFD report is as of, or for ...",0,1,1
317,Outcome: The bank explained that it would be w...,1,1,1
318,"In 2020, Banco do Brasil Foundation celebrated...",0,1,1


In [ ]:
df['gpt-label'].value_counts()

 1    272
 0     44
-1      4
Name: gpt-label, dtype: int64

In [ ]:
df[df['gpt-label']==-1]

,text,label,gpt-label,gpt-explanations
18, To exclude the least energy efficient hydroc...,1,-1,climate protection projects certified by indep...
249,Although climate change has not yet had a mate...,0,-1,operations and has made a commitment to transi...
268,As we invest in Manufactured Capital to improv...,0,-1,and Human Capital by minimizing negative impac...
270,"Increasing population, urbanisation, our own i...",0,-1,"Technology, we can create new instruments and ..."


In [ ]:
df['gpt-label'] = df['gpt-label'].replace(-1,1)

In [ ]:
df['gpt-label'].value_counts()

1    276
0     44
Name: gpt-label, dtype: int64

In [ ]:
df.to_csv("/content/drive/MyDrive/DS-Environment-Project/ChatGPT Results/chatgpt_climate_specificity.csv",index=False)

In [ ]:
# calculate the precision and f1 score for df columns label and prediction
from sklearn.metrics import precision_recall_fscore_support,f1_score
sent_col = 'gpt-label'
print(precision_recall_fscore_support(df['label'], df[sent_col], average='macro'))

# f1 score only
print(f1_score(df['label'], df[sent_col], average='macro'))

# confusion matrix
from sklearn.metrics import confusion_matrix
print(confusion_matrix(df['label'], df[sent_col]))

# performnce report

from sklearn.metrics import classification_report
print(classification_report(df['label'], df[sent_col]))

(0.3934453227931489, 0.4465973339936445, 0.32445220936157826, None)
0.32445220936157826
[[ 19 178]
 [ 25  98]]
              precision    recall  f1-score   support

           0       0.43      0.10      0.16       197
           1       0.36      0.80      0.49       123

    accuracy                           0.37       320
   macro avg       0.39      0.45      0.32       320
weighted avg       0.40      0.37      0.29       320



# testing Climate Classification with TCFD recommendations on ChatGPT
## Recommended classes:
### 0 - not climate-related
### 1 - metrics
### 2 - strategy
### 3 - risk
### 4 - governance


In [ ]:
!pip install datasets
from datasets import load_dataset

dataset = load_dataset("climatebert/tcfd_recommendations")

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


Extracting data files:   0%|          | 0/2 [00:00<?, ?it/s]

Generating train split:   0%|          | 0/1300 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/400 [00:00<?, ? examples/s]

Dataset parquet downloaded and prepared to /root/.cache/huggingface/datasets/climatebert___parquet/climatebert--tcfd_recommendations-8f7123f770abbd61/0.0.0/2a3b91fbd88a2c90d1dbbb32b460cf621d31bd5b05b934492fdef7d8d6f236ec. Subsequent calls will reuse this data.


  0%|          | 0/2 [00:00<?, ?it/s]

In [ ]:
dataset

DatasetDict({
    train: Dataset({
        features: ['text', 'label'],
        num_rows: 1300
    })
    test: Dataset({
        features: ['text', 'label'],
        num_rows: 400
    })
})

In [ ]:
dataset['test']

Dataset({
    features: ['text', 'label'],
    num_rows: 400
})

In [ ]:
data = []
for i in range(0,len(dataset['test']['text'])):
  data.append([dataset['test']['text'][i],dataset['test']['label'][i]])

print(data)

[['Sustainable strategy ‘red lines’ For our sustainable strategy range, we incorporate a series of proprietary ‘red lines’ in order to ensure the poorest- performing companies from an ESG perspective are not eligible for investment.', 2], ['Verizon’s environmental, health and safety management system provides a framework for identifying, controlling, and reducing the risks associated with the environments in which we operate. Besides regular management system assessments, internal and third-party compliance audits and inspections are performed annually at hundreds of facilities worldwide. The goal of these assessments is to identify and correct site-specific issues, and to educate and empower facility managers and supervisors to implement corrective actions. Verizon’s environment, health and safety efforts are directed and supported by experienced experts around the world that support our operations and facilities.', 3], ['In 2019, the Company closed a series of transactions related to

In [ ]:
import pandas as pd
df = pd.DataFrame(data=data,columns=["text","label"])

In [ ]:
df

,text,label
0,Sustainable strategy ‘red lines’ For our susta...,2
1,"Verizon’s environmental, health and safety man...",3
2,"In 2019, the Company closed a series of transa...",2
3,"In December 2020, the AUC approved the Electri...",2
4,"Finally, there is a reputational risk linked t...",2
...,...,...
395,"In 2020, Banco do Brasil Foundation celebrated...",2
396,Climate change is producing changes in weather...,2
397,A sound and certain regulatory and fiscal envi...,0
398,"Across our global workforce, 20% of Gold Field...",0


In [ ]:
prompt = 'You are the sustainability, environment, and climate change expert. Is the following text about metrics, strategy, risk, governance or is not climate change-related? Answer only with 0 if the text is not climate-related, answer only with 1 if the text is about metrics for sustainability, environment, and climate change, answer only with 2 if the text is about strategy for sustainability, environment, and climate change, answer only with 3 if the text is about risk for sustainability, environment, and climate change and answer only with 4 if the text is about governance for sustainability, environment, and climate change: \n\n'
texts = df["text"].to_list()
rez, rez_keys = batch_gpt_len(prompt, texts, 1)

1 860 - 2 2: error calling API, retrying...
2 1318 - 3 3: 3 1247 - 1 1: 4 1560 - 0 0: 5 1050 - 3 3: 6 1019 - 1 1: 7 1016 - 3 - risk for sustainability, environment, and climate change. 3: 8 1064 - 0 0: 9 1160 - 4 4: 10 935 - 3 - The text is about risk for sustainability, environment, and climate change. 3: 
11 1224 - 4 4: 12 891 - 3 - risk for sustainability, environment, and climate change. 3: 13 903 - 1 1: 14 1137 - 3 3: 15 857 - 1 1: 16 1066 - 0 0: error parsingAnswer: 1 (metrics for sustainability and energy conservation initiatives)
17 916 - Answer: 1 (metrics for sustainability and energy conservation initiatives) -1: 18 1077 - 1 (metrics for sustainability, environment, and climate change) 1: 19 1000 - 3 3: 20 831 - 3 3: 
21 1307 - 3 3: 22 1066 - 0 0: error parsingfollowing actions: 

3. This text is about risk for sustainability, environment, and climate change. It highlights the risk associated with investing in hydrocarbons that are least energy efficient and pose a threat to

In [ ]:
rez

['2',
 '3',
 '1',
 '0',
 '3',
 '1',
 '3 - risk for sustainability, environment, and climate change.',
 '0',
 '4',
 '3 - The text is about risk for sustainability, environment, and climate change.',
 '4',
 '3 - risk for sustainability, environment, and climate change.',
 '1',
 '3',
 '1',
 '0',
 'Answer: 1 (metrics for sustainability and energy conservation initiatives)',
 '1 (metrics for sustainability, environment, and climate change)',
 '3',
 '3',
 '3',
 '0',
 "following actions: \n\n3. This text is about risk for sustainability, environment, and climate change. It highlights the risk associated with investing in hydrocarbons that are least energy efficient and pose a threat to the environment, which is incompatible with the goal of combating climate change and represents an economic risk for investors. It further discusses the exclusion criteria set out to turn down projects and companies that do business in such hydrocarbons. Additionally, it mentions Crédit Agricole S.A.'s commitme

In [ ]:
rez_keys

[2,
 3,
 1,
 0,
 3,
 1,
 3,
 0,
 4,
 3,
 4,
 3,
 1,
 3,
 1,
 0,
 -1,
 1,
 3,
 3,
 3,
 0,
 -1,
 1,
 3,
 3,
 2,
 1,
 3,
 3,
 1,
 3,
 2,
 3,
 1,
 1,
 2,
 3,
 3,
 2,
 -1,
 1,
 2,
 1,
 1,
 1,
 4,
 3,
 4,
 0,
 4,
 1,
 3,
 3,
 2,
 2,
 3,
 1,
 0,
 1,
 1,
 2,
 3,
 3,
 1,
 2,
 0,
 3,
 1,
 3,
 3,
 0,
 -1,
 1,
 3,
 3,
 1,
 2,
 3,
 0,
 3,
 1,
 0,
 1,
 4,
 2,
 2,
 2,
 3,
 2,
 3,
 3,
 1,
 3,
 3,
 3,
 1,
 1,
 1,
 1,
 1,
 1,
 0,
 0,
 1,
 2,
 3,
 4,
 1,
 3,
 3,
 0,
 3,
 1,
 1,
 3,
 3,
 3,
 0,
 0,
 4,
 1,
 0,
 0,
 1,
 3,
 3,
 3,
 3,
 3,
 4,
 2,
 1,
 3,
 4,
 2,
 1,
 3,
 4,
 2,
 1,
 3,
 3,
 3,
 3,
 3,
 3,
 4,
 4,
 2,
 3,
 3,
 3,
 2,
 1,
 1,
 2,
 1,
 1,
 0,
 1,
 4,
 3,
 1,
 3,
 2,
 1,
 3,
 3,
 1,
 4,
 0,
 2,
 2,
 1,
 3,
 1,
 3,
 3,
 3,
 4,
 0,
 4,
 1,
 1,
 2,
 2,
 1,
 3,
 -1,
 3,
 1,
 2,
 2,
 4,
 1,
 1,
 4,
 1,
 4,
 2,
 4,
 3,
 3,
 0,
 3,
 3,
 3,
 1,
 1,
 4,
 1,
 4,
 3,
 3,
 2,
 4,
 0,
 3,
 -1,
 1,
 4,
 4,
 3,
 1,
 1,
 2,
 0,
 2,
 3,
 1,
 4,
 3,
 3,
 3,
 2,
 0,
 3,
 4,
 3,
 1,
 4,
 2,
 3,
 2,
 0,
 2,
 2,
 4

In [ ]:
df['gpt-label'] = rez_keys

In [ ]:
df['gpt-explanations'] = rez

In [ ]:
df

,text,label,gpt-label,gpt-explanations
0,Sustainable strategy ‘red lines’ For our susta...,2,2,2
1,"Verizon’s environmental, health and safety man...",3,3,3
2,"In 2019, the Company closed a series of transa...",2,1,1
3,"In December 2020, the AUC approved the Electri...",2,0,0
4,"Finally, there is a reputational risk linked t...",2,3,3
...,...,...,...,...
395,"In 2020, Banco do Brasil Foundation celebrated...",2,-1,Not climate change-related (0)
396,Climate change is producing changes in weather...,2,3,3
397,A sound and certain regulatory and fiscal envi...,0,4,4
398,"Across our global workforce, 20% of Gold Field...",0,0,0


In [ ]:
df['gpt-label'].value_counts()

 3    124
 1    105
 2     57
 0     54
 4     40
-1     20
Name: gpt-label, dtype: int64

In [ ]:
df[df['gpt-label']==-1]

,text,label,gpt-label,gpt-explanations,-1
326,"On January 5, 2021, Centrica plc. closed a tra...",2,-1,Answer: 3 (risk),3


In [ ]:
list_3 = [22,40,72,189,219,287,301,302,326,354,362,364]
list_0 = [289,395]
list_1 = [16]
list_2 = [264,282,323]
list_4 = [317,366]

In [ ]:
for i in list_0:
  df['gpt-label'][i] = 0

for i in list_1:
  df['gpt-label'][i] = 1

for i in list_2:
  df['gpt-label'][i] = 2

for i in list_3:
  df['gpt-label'][i] = 3

for i in list_4:
  df['gpt-label'][i] = 4

<ipython-input-147-c908468fb7a4>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['gpt-label'][i] = 0
<ipython-input-147-c908468fb7a4>:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['gpt-label'][i] = 1
<ipython-input-147-c908468fb7a4>:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['gpt-label'][i] = 2
<ipython-input-147-c908468fb7a4>:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFr

In [ ]:
df.iloc[[22,40,72,189,219,287,289,301,302,354,362,364]]

,text,label,gpt-label,gpt-explanations,-1
22, To exclude the least energy efficient hydroc...,2,3,following actions: \n\n3. This text is about r...,3
40,(and the company may not be able to offset suc...,2,3,"Answer: 3 (Risk for sustainability, environmen...",3
72,"THE ISSUE Investors, in our view, have been to...",2,3,"Answer: 3 (risk for sustainability, environmen...",3
189,There are many projections about the future gr...,2,3,"Answer: 3 (risk for sustainability, environmen...",3
219,G. Operational risk Operational risk definitio...,0,3,"Answer: 3 (risk for sustainability, environmen...",3
287,Our ability to maintain profitability in an in...,0,3,Answer: 3,3
289,"The wholesale banking sector is a mature, high...",0,0,Not climate change-related.,3
301,Although climate change has not yet had a mate...,2,3,operations and has therefore been enhancing it...,3
302,Nedbank has a Damage to Physical Assets scenar...,3,3,"The text is about risk for sustainability, env...",3
354,"Over the past several years, changing weather ...",2,3,Answer: 3,3


In [ ]:
df['gpt-label'].value_counts()

3    136
1    106
2     60
0     56
4     42
Name: gpt-label, dtype: int64

In [ ]:
df.to_csv("/content/drive/MyDrive/DS-Environment-Project/ChatGPT Results/chatgpt_climate_tcfd_recommendations.csv",index=False)

In [ ]:
# calculate the precision and f1 score for df columns label and prediction
from sklearn.metrics import precision_recall_fscore_support,f1_score
sent_col = 'gpt-label'
print(precision_recall_fscore_support(df['label'], df[sent_col], average='macro'))

# f1 score only
print(f1_score(df['label'], df[sent_col], average='macro'))

# confusion matrix
from sklearn.metrics import confusion_matrix
print(confusion_matrix(df['label'], df[sent_col]))

# performnce report

from sklearn.metrics import classification_report
print(classification_report(df['label'], df[sent_col]))

(0.4383507742719729, 0.49523122212571097, 0.39938956315003993, None)
0.39938956315003993
[[35  2  5 25 13]
 [ 4 38  4  2  1]
 [16 55 43 72 11]
 [ 1  9  5 28  5]
 [ 0  2  3  9 12]]
              precision    recall  f1-score   support

           0       0.62      0.44      0.51        80
           1       0.36      0.78      0.49        49
           2       0.72      0.22      0.33       197
           3       0.21      0.58      0.30        48
           4       0.29      0.46      0.35        26

    accuracy                           0.39       400
   macro avg       0.44      0.50      0.40       400
weighted avg       0.57      0.39      0.39       400

